<a href="https://www.kaggle.com/code/dorivankadatz/time-series-store-sales-forecasting?scriptVersionId=110225680" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div>
    <a id="mainmenu"></a>
</div>

# Store Sales - Time Series Forecasting
Author: Dorivan Kadatz Borba<br/>
Coding: utf-8

#### Kagle Competition Link: https://www.kaggle.com/competitions/store-sales-time-series-forecasting/overview

### We need to forecast grocery sales, using time-series store sales data from Corporación Favorita, a large Ecuadorian-based grocery retailer.

### Contents
- [01. Initial libraries](#import)
- [02. Loading Datasets](#importingdata)
- [03. Pre Visualization](#previsualization)
   - [03.1. Insights](#previsualizationinsight)
- [04. Feature Engeneering](#features)
- [05. Train / Test Split](#split)
- [06. First Model Xboost](#firstxboost)
   - [06.1. Forecast](#fore1)
   - [06.2. Insights](#model1insight)
- [07. Grid Search](#gridcross)
- [08. Model with the best params from Grid Search](#bestparamsmodel)
   - [08.1. Forecast](#fore2)
   - [06.2. Insights](#model1insight2)
- [09. Preparing Submission Data](#submission)
- [10. Train Full Data Set to Submission](#submission2)

# 01) Initial libraries
- [Back to the contents](#mainmenu)
<div>
    <a id="import"></a>
</div>

In [1]:
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# During the notebook I will install other libraries and python functions,
# maybe this will facilitate understanding.

<div>
    <a id="importingdata"></a>
</div>

# 02) Loading datasets
- [Back to the contents](#mainmenu)

In [2]:
# Data file path
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Loading data
train_df = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
holidays_df = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
test_df = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')
sample_submission = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


<div>
    <a id="previsualization"></a>
</div>

# 03) Pre Vizualization
- [Back to the contents](#mainmenu)

In [3]:
# Seeing the face of data set
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [4]:
# Pandas Profile, show us a resume of dataset
from pandas_profiling import ProfileReport

# The pandas profile does not work well on the github, just uncomment the line to run.

#ProfileReport(train_df)

In [5]:
# Transform the type of 'date' to date time and set to index
train_df['date'] = pd.to_datetime(train_df['date'])
train_df.set_index('date', inplace=True)

In [6]:
# How is the variables types
train_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3000888 entries, 2013-01-01 to 2017-08-15
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   store_nbr    int64  
 2   family       object 
 3   sales        float64
 4   onpromotion  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 137.4+ MB


In [7]:
# Return the sum sales by day
import matplotlib.pyplot as plt
sales_day = pd.DataFrame(train_df['sales'].resample('D').sum())

sales_day['sales'].plot(figsize=(15,5))
plt.title('Corporación Favorita Sales by Day', fontsize=25)
plt.xlabel('Time', fontsize=15)
plt.ylabel('Sales', fontsize=15)
plt.show()

In [8]:
sales_day = sales_day.loc[(sales_day.index >= '2016-12-23') & (sales_day.index <= '2017-01-02')]

sales_day['sales'].plot(figsize=(15,5))
plt.title('Corporación Favorita Sales 2016-12-23 to 2017-01-02', fontsize=25)
plt.xlabel('Time', fontsize=15)
plt.ylabel('Sales', fontsize=15)
plt.show()

In [9]:
sales_day.loc[sales_day.index == '2016-12-25']

,sales
date,
2016-12-25,0.0


In [10]:
sales_day.loc[sales_day.index == '2017-01-01']

,sales
date,
2017-01-01,12082.500997


In [11]:
train_df.loc[train_df.index == '2016-01-01'].sum()

id                                                    3469259079
store_nbr                                                  49005
family         AUTOMOTIVEBABY CAREBEAUTYBEVERAGESBOOKSBREAD/B...
sales                                                  16433.394
onpromotion                                                  159
dtype: object

In [12]:
train_df.loc[train_df.index == '2015-12-25'].sum()

id             0.0
store_nbr      0.0
family         0.0
sales          0.0
onpromotion    0.0
dtype: float64

<div>
    <a id="previsualizationinsight"></a>
</div>

### Insights
##### - We have a time series with some variables, wich each products family depends on each store. <br> - The Corporación Favorita have 54 stores around of Ecuador, with 33 unique type of product family. <br> - The time of this dataset goes to 2013-01-01 to 2017-08-05. <br> - So, each day have 1782 sales obervations. <br> - This series doesn't have null values and duplicate values.<br> - The profile tell us there are a positive correlation between 'sales' and 'promotions'. <br> - All the stores are closed in the Christmas so the sales are zero, and first day of year we have some stores opend.

<div>
    <a id="features"></a>
</div>

# 04) Feature Engeneering
- [Back to the contents](#mainmenu)

In [13]:
holidays_df['date'] = pd.to_datetime(holidays_df['date'])
holidays_df.set_index('date', inplace=True)

In [14]:
holidays_df.head()

,type,locale,locale_name,description,transferred
date,,,,,
2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [15]:
%%time

# TIME FEATURES

# XGBoost is a supervised learning process to work with, we are required to make our data data
# for supervised learning.
# RF: https://analyticsindiamag.com/how-to-use-xgboost-for-time-series-analysis/#:~:text=XGBoost%20is%20an%20efficient%20technique%20for%20implementing%20gradient,considered%20as%20an%20advancement%20of%20traditional%20modelling%20techniques.
def create_features(df):
    df['date'] = df.index
    #df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    return df

# Add the time features to our data frame
train_df = create_features(train_df)

#
#
#

# LAG FEATURES

'''We shift the values to see the past  sales by 1 day, 21day...
ex: the same product family sales today on 'sales' variable, and in the same row how much are saled
yesterday, three weeks before...
'''
def lag_features(df):
    # Return a list of all sales observations
    target_map = pd.DataFrame(df['sales'])
    df['lag16days'] = target_map.shift(28512)
    df['lag21days'] = target_map.shift(37422)
    df['lag28days'] = target_map.shift(49896)
    return df

# Add the lag features to our data frame
train_df = lag_features(train_df)

#
#
#

# HOLIDAYS FEATURES

# Join Holidays_df on train_df
train_df = train_df.join(holidays_df)

# It is an important part of data preprocessing to encode labels appropriately in numerical form
# in order to make sure that the learning algorithm interprets the features correctly
# Ref: https://vitalflux.com/labelencoder-example-single-multiple-columns/
from sklearn.preprocessing import LabelEncoder

# LABEL ENCODER

# Replace our variable categoric to numeric representation
train_df[['family',
          'type',
          'locale',
          'locale_name',
          'description',
          'transferred']] = train_df[['family',
                                      'type',
                                      'locale',
                                      'locale_name',
                                      'description',
                                      'transferred']].apply(LabelEncoder().fit_transform)


# Drop column 'date' and 'id'
train_df.drop(['id', 'date', 'transferred'], axis=1, inplace=True)
train_df.dropna(inplace=True)

CPU times: user 6.68 s, sys: 392 ms, total: 7.08 s
Wall time: 7.12 s


In [16]:
train_df

,store_nbr,family,sales,onpromotion,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,lag16days,lag21days,lag28days,type,locale,locale_name,description
date,,,,,,,,,,,,,,,,,,
2013-01-29,1,0,2.000,0,1,1,1,2013,29,29,5,2.000,2.000,0.000,6,3,24,101
2013-01-29,1,1,0.000,0,1,1,1,2013,29,29,5,0.000,0.000,0.000,6,3,24,101
2013-01-29,1,2,3.000,0,1,1,1,2013,29,29,5,0.000,3.000,0.000,6,3,24,101
2013-01-29,1,3,932.000,0,1,1,1,2013,29,29,5,572.000,1029.000,0.000,6,3,24,101
2013-01-29,1,4,0.000,0,1,1,1,2013,29,29,5,0.000,0.000,0.000,6,3,24,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-15,9,28,438.133,0,1,3,8,2017,227,15,33,517.911,320.009,320.401,3,0,19,28
2017-08-15,9,29,154.553,1,1,3,8,2017,227,15,33,145.490,51.879,118.927,3,0,19,28
2017-08-15,9,30,2419.729,148,1,3,8,2017,227,15,33,1882.588,2100.046,2178.149,3,0,19,28


<div>
    <a id="split"></a>
</div>

# 05) Train / Test Split
- [Back to the contents](#mainmenu)

In [17]:
# Slice the 'train_df' to new variables 'train' and 'test'
train = train_df.loc[train_df.index < '2016-08-01']

# Final test data set
test = train_df.loc[train_df.index >= '2016-08-01']

In [18]:
print(f'''train shape: {train.shape}
test shape: {test.shape}
Percent to Test: {round(test.shape[0] / train.shape[0]*100, 2)}''')

train shape: (2318382, 18)
test shape: (686070, 18)
Percent to Test: 29.59


In [19]:
# Plot a graphic with the train and test split
plt.style.use('seaborn')

fig, ax = plt.subplots(figsize=(15,5))
train['sales'].plot(ax=ax,label='Training Set')
test['sales'].plot(ax=ax, label='Test Set')
plt.title('Corporación Favorita Individual Store Sales', fontsize=25)
plt.xlabel('Time', fontsize=15)
plt.ylabel('Sales', fontsize=15)
plt.legend(fontsize=15)
plt.show()

<div>
    <a id="firstxboost"></a>
</div>

# 06) First Model Xboost
- [Back to the contents](#mainmenu)

In [20]:
%%time
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# Cross Validation with TimeSeriesSplit with 3 folds
tss = TimeSeriesSplit(n_splits=3)
   
preds = []
scores = []
for train_index, val_index in tss.split(train):
    train_s = train.iloc[train_index]
    test_s = train.iloc[val_index]
    
    # Split the target from features to train data set
    X_train = train_s.drop('sales', axis=1)
    y_train = train_s['sales']
    
    # Split the target from features to test data set
    X_test = test_s.drop('sales', axis=1)
    y_test = test_s['sales']
    
    # Creates the regressor model
    reg_model = XGBRegressor(n_estimators=500,
                             booster = 'gbtree',
                             objective= 'reg:squarederror',
                             early_stopping_rounds=50,)

    # Train the model
    reg_model.fit(X_train, y_train,
                 eval_set=[(X_train, y_train), (X_test, y_test)],
                 verbose=200)

    y_pred = reg_model.predict(X_test)
    preds.append(y_pred)
    score = np.sqrt(mean_squared_error(y_test, y_pred))
    scores.append(score)

[0]	validation_0-rmse:541.23132	validation_1-rmse:766.07296
[195]	validation_0-rmse:78.34194	validation_1-rmse:458.97442
[0]	validation_0-rmse:659.21387	validation_1-rmse:848.12036
[61]	validation_0-rmse:183.92602	validation_1-rmse:376.48143
[0]	validation_0-rmse:721.56575	validation_1-rmse:1040.81323
[100]	validation_0-rmse:199.10512	validation_1-rmse:473.05522
CPU times: user 15min 43s, sys: 1.05 s, total: 15min 44s
Wall time: 4min 1s


In [21]:
print(f'Score across folds: {np.mean(scores):0.2f}')
print(f'Fold scores: {scores}')

Score across folds: 425.96
Fold scores: [458.8176457384987, 349.81869845810144, 469.2411802237608]


<div>
    <a id="fore1"></a>
</div>


### Forecast

In [22]:
%%time
# New variable on our 'test' data frame with the predicion value from 'X_test'
# Split the target from features to train data set
X_train = train.drop('sales', axis=1)
y_train = train['sales']
    
# Split the target from features to test data set
X_test = test.drop('sales', axis=1)
y_test = test['sales']

test['prediction'] = reg_model.predict(X_test)

CPU times: user 1.3 s, sys: 47.9 ms, total: 1.35 s
Wall time: 406 ms


In [23]:
# Statistics sales vs prediction
test[['sales', 'prediction']].describe()

,sales,prediction
count,686070.000000,686070.000000
mean,471.259719,480.384247
std,1345.605474,1280.256958
min,0.000000,-1902.382935
25%,3.000000,6.673768
50%,25.000000,52.475273
75%,267.770250,290.712357
max,89576.360000,17618.406250


In [24]:
test[['sales', 'prediction' ]].query('prediction < 0')

,sales,prediction
date,,
2016-08-01,0.0,-0.264731
2016-08-01,0.0,-1.296335
2016-08-01,0.0,-0.126505
2016-08-01,0.0,-0.469706
2016-08-01,0.0,-0.469706
...,...,...
2017-08-15,0.0,-0.935030
2017-08-15,2.0,-0.327500
2017-08-15,0.0,-0.935030


In [25]:
sales_day = pd.DataFrame(test[['sales', 'prediction']].resample('D').sum())

fig, ax = plt.subplots(figsize=(15,5))
sales_day['sales'].plot(ax=ax, label='True Value')
sales_day['prediction'].plot(ax=ax, label='Prediction')
plt.title('Truth value vs Prediction value Total Sales by Day', fontsize=25)
plt.legend(fontsize=15)
plt.xlabel('Time', fontsize=15)
plt.ylabel('Sales', fontsize=15)
plt.show()

In [26]:
fi = pd.DataFrame(data= reg_model.feature_importances_,
             index=reg_model.feature_names_in_,
             columns=['importance'])
fi

,importance
store_nbr,0.005438
family,0.024535
onpromotion,0.012619
dayofweek,0.017761
quarter,0.021262
month,0.017383
year,0.020413
dayofyear,0.016406
dayofmonth,0.015843
weekofyear,0.024440


In [27]:
fi.sort_values(by='importance').plot(kind='barh', title = 'Features Importance')

<AxesSubplot:title={'center':'Features Importance'}>

In [28]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_log_error

def timeseries_evaluation_metrics(y_true, y_pred):
    print('Evaluation metric results: ')
    print(f'MSE: {mean_squared_error(y_true, y_pred)}')
    print(f'MAE: {mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE: {np.sqrt(mean_squared_error(y_true, y_pred))}')
    print(f'RMSLE: {np.sqrt(mean_squared_log_error( y_true, np.abs(y_pred)))}')
    print(f'MAPE: {mean_absolute_percentage_error(y_true, y_pred)}')
    print(f'R2: {r2_score(y_true, y_pred)}', end='\n\n')
    

In [29]:
timeseries_evaluation_metrics(test['sales'], test['prediction'])

Evaluation metric results: 
MSE: 140795.2304057481
MAE: 103.49731046954608
RMSE: 375.22690522635514
RMSLE: 1.145434676771235
MAPE: 1.1124964306280746e+16
R2: 0.9222405669367715



<div>
    <a id="model1insight"></a>
</div>

### Insights:
##### - In this model we used a dataset with holidays features,  let's see the 'describe', the min prediction should be near by 0, but in some cases the value goes to 1900 negative, the max real sales is 89k vs prediction with 17.6k, the diference is giant, worth remembering, these peaks are rare and difficult to reach .<br>-RMSLE: 1.14 <br>-RMSE: 375.22

###  let's try to improve the model from here...

<div>
    <a id="gridcross"></a>
</div>

# 07) Grid Search
- [Back to the contents](#mainmenu)


%%time
from sklearn.model_selection import GridSearchCV

tss = TimeSeriesSplit(n_splits=3)

reg_model = XGBRegressor()
parameters = {'booster':['gbtree'],
              'objective': ['reg:squarederror'],
              'learning_rate': [0.01, 0.02, 0.03],
              'max_depth': [7, 8, 9],
              'min_child_weight': [4, 5, 6],
              'subsample': [0.5, 0.7, 1],
              'colsample_bytree': [0.5, 0.7, 1],
              'n_estimators': [500],
              'verbose': [500]}

reg_model_grid = GridSearchCV(reg_model,
                        parameters,
                        cv = tss)

reg_model_grid.fit(X_train, y_train)

print(grid_search_result.best_score_)
print(grid_search_result.best_params_)

##### These cells is markdown because we have the result, do not need to run again, this run lasted some hours. <br>Double-click the cells to see the script.

<div>
    <a id="bestparamsmodel"></a>
</div>

# 08) Model with the best params from Grid Search
- [Back to the contents](#mainmenu)

In [30]:
%%time
# Creates the regressor model
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
tss = TimeSeriesSplit(n_splits=5)

preds = []
scores = []
for train_index, val_index in tss.split(train):
    train_s = train.iloc[train_index]
    test_s = train.iloc[val_index]
    
    # Split the target from features to train data set
    X_train = train_s.drop('sales', axis=1)
    y_train = train_s['sales']
    
    # Split the target from features to test data set
    X_test = test_s.drop('sales', axis=1)
    y_test = test_s['sales']
    
    # Creates the regressor model
    reg_model = XGBRegressor(booster = 'gbtree',
                             objective= 'reg:squarederror',
                             early_stopping_rounds=50,
                             colsample_bytree= 0.7,
                             subsample= 0.7,
                             learning_rate= 0.01,
                             max_depth= 8,
                             min_child_weight= 4,
                             n_estimators=500)

    # Train the model
    reg_model.fit(X_train, y_train,
                 eval_set=[(X_train, y_train), (X_test, y_test)],
                 verbose=250)

    y_pred = reg_model.predict(X_test)
    preds.append(y_pred)
    score = np.sqrt(mean_squared_error(y_test, y_pred))
    scores.append(score)

[0]	validation_0-rmse:713.34686	validation_1-rmse:927.03449
[250]	validation_0-rmse:128.20496	validation_1-rmse:426.84978
[499]	validation_0-rmse:98.31376	validation_1-rmse:416.23403
[0]	validation_0-rmse:826.68217	validation_1-rmse:967.41974
[250]	validation_0-rmse:213.96505	validation_1-rmse:356.67381
[499]	validation_0-rmse:168.25143	validation_1-rmse:344.12150
[0]	validation_0-rmse:876.05021	validation_1-rmse:1055.40204
[250]	validation_0-rmse:248.07623	validation_1-rmse:324.64920
[444]	validation_0-rmse:207.38599	validation_1-rmse:308.92667
[0]	validation_0-rmse:924.14855	validation_1-rmse:1297.06594
[250]	validation_0-rmse:261.82631	validation_1-rmse:396.78694
[499]	validation_0-rmse:216.61684	validation_1-rmse:368.26798
[0]	validation_0-rmse:1009.63986	validation_1-rmse:1348.96648
[250]	validation_0-rmse:280.66451	validation_1-rmse:497.87581
[499]	validation_0-rmse:233.82003	validation_1-rmse:466.80094
CPU times: user 2h 9min 29s, sys: 6 s, total: 2h 9min 35s
Wall time: 33min 57

In [31]:
# Scores of the model with best params found by grid search
print(f'Score across folds: {np.mean(scores):0.2f}')
print(f'Fold scores: {scores}')

Score across folds: 380.80
Fold scores: [416.2340328845097, 344.06549688027235, 308.7726169673876, 368.112489114199, 466.80094016325546]


<div>
    <a id="fore2"></a>
</div>


### Forecast

In [32]:
%%time
X_test = test.drop(['prediction','sales'], axis=1)
y_test = test['sales']

# Add new variable and predict the values
test['prediction'] = reg_model.predict(X_test)

CPU times: user 16.5 s, sys: 27 ms, total: 16.5 s
Wall time: 4.19 s


In [33]:
test[['sales', 'prediction']].describe()

,sales,prediction
count,686070.000000,686070.000000
mean,471.259719,479.536499
std,1345.605474,1272.178589
min,0.000000,-13.106738
25%,3.000000,4.799524
50%,25.000000,49.601959
75%,267.770250,289.015831
max,89576.360000,17235.324219


In [34]:
# Return the days with negative prediction
test[['sales', 'prediction' ]].query('prediction < 0')

,sales,prediction
date,,
2016-08-07,0.0,-0.451091
2016-08-14,0.0,-0.280210
2016-08-17,1.0,-0.259380
2016-08-17,0.0,-0.259380
2016-08-17,0.0,-0.159478
...,...,...
2017-07-31,0.0,-0.214552
2017-07-31,0.0,-0.399585
2017-07-31,1.0,-0.599747


In [35]:
sales_day = pd.DataFrame(test[['sales', 'prediction']].resample('D').sum())

fig, ax = plt.subplots(figsize=(15,5))
sales_day['sales'].plot(ax=ax, label='True Value')
sales_day['prediction'].plot(ax=ax, label='Prediction')
plt.title('Truth value vs Prediction value Total Sales by Day', fontsize=25)
plt.legend(fontsize=15)
plt.xlabel('Time', fontsize=15)
plt.ylabel('Sales', fontsize=15)
plt.show()

In [36]:
timeseries_evaluation_metrics(test['sales'], test['prediction'])

Evaluation metric results: 
MSE: 108663.13427962911
MAE: 87.06715559701115
RMSE: 329.64091718054226
RMSLE: 0.9448199091773178
MAPE: 8782129070623607.0
R2: 0.939986719066355



In [37]:
# Let's see some samples to compare the sales and prediction
test[['store_nbr', 'family','sales', 'prediction' ]].sample(20)

,store_nbr,family,sales,prediction
date,,,,
2017-02-20,33,9,136.000,171.409882
2017-03-20,35,6,2.000,0.300705
2017-06-18,42,26,11.000,9.829282
2017-08-08,1,21,16.000,8.574152
2016-09-11,12,13,2.000,48.263638
2017-02-24,13,29,46.000,49.981895
2016-09-29,13,13,1.000,0.920000
2016-08-23,42,24,188.357,169.286346
2017-02-19,44,28,1294.149,1826.848755


<div>
    <a id="model1insight2"></a>
</div>

### Insights:
##### - we haven't had a significant improvement with the tunning model, but the min prediction decreased to 9 negative muahahahaha,<br><br> - R2: 0.94<br>- RMLSE: 0.91<br><br>- Of course, this is not the best model from kaggle with this data set, I'm really happy with the result for my fisrt machine learning project, I'll try to get a best metrics later.


<div>
    <a id="submission"></a>
</div>

# 09) Preparing Submission Data
- [Back to the contents](#mainmenu)

In [38]:
train_df2 = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')

In [39]:
test_df

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [40]:
train_df2.head(2)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0


In [41]:
# Transform the type of 'date' to date time and set to index
test_df['date'] = pd.to_datetime(test_df['date'])
test_df.set_index('date', inplace=True)

train_df2['date'] = pd.to_datetime(train_df2['date'])
train_df2.set_index('date', inplace=True)

In [42]:
# Return a slice with last 28 days to create the lag features to predicti the submission data set
temp_df = train_df2[-49896:]

In [43]:
# Concat train_df and temp_df to create submission_df
submission_df = pd.concat([temp_df, test_df], axis=0)

In [44]:
# I need this dataframe to create the features with lags, then I'll split again after to get only
# submission dataframe to predict
submission_df

,id,store_nbr,family,sales,onpromotion
date,,,,,
2017-07-19,2950992,1,AUTOMOTIVE,7.0,0
2017-07-19,2950993,1,BABY CARE,0.0,0
2017-07-19,2950994,1,BEAUTY,3.0,0
2017-07-19,2950995,1,BEVERAGES,2369.0,34
2017-07-19,2950996,1,BOOKS,0.0,0
...,...,...,...,...,...
2017-08-31,3029395,9,POULTRY,NaN,1
2017-08-31,3029396,9,PREPARED FOODS,NaN,0
2017-08-31,3029397,9,PRODUCE,NaN,1


###### Feature Engeneering

In [45]:
temp_df.index.unique()

DatetimeIndex(['2017-07-19', '2017-07-20', '2017-07-21', '2017-07-22',
               '2017-07-23', '2017-07-24', '2017-07-25', '2017-07-26',
               '2017-07-27', '2017-07-28', '2017-07-29', '2017-07-30',
               '2017-07-31', '2017-08-01', '2017-08-02', '2017-08-03',
               '2017-08-04', '2017-08-05', '2017-08-06', '2017-08-07',
               '2017-08-08', '2017-08-09', '2017-08-10', '2017-08-11',
               '2017-08-12', '2017-08-13', '2017-08-14', '2017-08-15'],
              dtype='datetime64[ns]', name='date', freq=None)

In [46]:
%%time

# TIME FEATURES

# Add the time features to our data frame
submission_df = create_features(submission_df)

#
#
#

# LAG FEATURES

# Add the lag features to our data frame
submission_df = lag_features(submission_df)

#
#
#

# HOLIDAYS FEATURES

# Join Holidays_df on train_df
submission_df = submission_df.join(holidays_df)

# It is an important part of data preprocessing to encode labels appropriately in numerical form
# in order to make sure that the learning algorithm interprets the features correctly
# Ref: https://vitalflux.com/labelencoder-example-single-multiple-columns/
from sklearn.preprocessing import LabelEncoder

# LABEL ENCODER

# Replace our variable categoric to numeric representation
submission_df[['family',
          'type',
          'locale',
          'locale_name',
          'description',
          'transferred']] = submission_df[['family',
                                      'type',
                                      'locale',
                                      'locale_name',
                                      'description',
                                      'transferred']].apply(LabelEncoder().fit_transform)


# Drop column 'date' and 'id'
submission_df.drop(['id', 'date', 'transferred'], axis=1, inplace=True)

CPU times: user 182 ms, sys: 1 ms, total: 183 ms
Wall time: 183 ms


In [47]:
# Get the test_df to predict
submission_df = submission_df.loc[submission_df.index >= '2017-08-16']
submission_df.drop(['sales'], axis=1, inplace=True)

In [48]:
submission_df

,store_nbr,family,onpromotion,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,lag16days,lag21days,lag28days,type,locale,locale_name,description
date,,,,,,,,,,,,,,,,,
2017-08-16,1,0,0,2,3,8,2017,228,16,33,8.000,2.00000,7.000,3,2,6,8
2017-08-16,1,1,0,2,3,8,2017,228,16,33,0.000,0.00000,0.000,3,2,6,8
2017-08-16,1,2,2,2,3,8,2017,228,16,33,3.000,3.00000,3.000,3,2,6,8
2017-08-16,1,3,20,2,3,8,2017,228,16,33,2414.000,2242.00000,2369.000,3,2,6,8
2017-08-16,1,4,0,2,3,8,2017,228,16,33,1.000,0.00000,0.000,3,2,6,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-31,9,28,1,3,3,8,2017,243,31,35,438.133,291.82098,425.946,3,2,6,8
2017-08-31,9,29,0,3,3,8,2017,243,31,35,154.553,111.93000,83.426,3,2,6,8
2017-08-31,9,30,1,3,3,8,2017,243,31,35,2419.729,1036.43900,1364.578,3,2,6,8


<div>
    <a id="submission2"></a>
</div>

# 10)  Train Full Data Set to Submission
- [Back to the contents](#mainmenu)


In [49]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3004452 entries, 2013-01-29 to 2017-08-15
Data columns (total 18 columns):
 #   Column       Dtype  
---  ------       -----  
 0   store_nbr    int64  
 1   family       int64  
 2   sales        float64
 3   onpromotion  int64  
 4   dayofweek    int64  
 5   quarter      int64  
 6   month        int64  
 7   year         int64  
 8   dayofyear    int64  
 9   dayofmonth   int64  
 10  weekofyear   int64  
 11  lag16days    float64
 12  lag21days    float64
 13  lag28days    float64
 14  type         int64  
 15  locale       int64  
 16  locale_name  int64  
 17  description  int64  
dtypes: float64(4), int64(14)
memory usage: 435.5 MB


In [50]:
%%time

preds = []
scores = []
for train_index, val_index in tss.split(train):
    train_s = train_df.iloc[train_index]
    test_s = train_df.iloc[val_index]
    
    # Split the target from features to train data set
    X_train = train_s.drop('sales', axis=1)
    y_train = train_s['sales']
    
    # Split the target from features to test data set
    X_test = test_s.drop('sales', axis=1)
    y_test = test_s['sales']
    
    # Creates the regressor model
    reg_model = XGBRegressor(booster = 'gbtree',
                             objective= 'reg:squarederror',
                             early_stopping_rounds=50,
                             colsample_bytree= 0.7,
                             subsample= 0.7,
                             learning_rate= 0.01,
                             max_depth= 8,
                             min_child_weight= 4,
                             n_estimators=500)

    # Train the model
    reg_model.fit(X_train, y_train,
                 eval_set=[(X_train, y_train), (X_test, y_test)],
                 verbose=250)

    y_pred = reg_model.predict(X_test)
    preds.append(y_pred)
    score = np.sqrt(mean_squared_error(y_test, y_pred))
    scores.append(score)

[0]	validation_0-rmse:713.34686	validation_1-rmse:927.03449
[250]	validation_0-rmse:128.20496	validation_1-rmse:426.84978
[499]	validation_0-rmse:98.31376	validation_1-rmse:416.23403
[0]	validation_0-rmse:826.68217	validation_1-rmse:967.41974
[250]	validation_0-rmse:213.96505	validation_1-rmse:356.67381
[499]	validation_0-rmse:168.25143	validation_1-rmse:344.12150
[0]	validation_0-rmse:876.05021	validation_1-rmse:1055.40204
[250]	validation_0-rmse:248.07623	validation_1-rmse:324.64920
[444]	validation_0-rmse:207.38599	validation_1-rmse:308.92667
[0]	validation_0-rmse:924.14855	validation_1-rmse:1297.06594
[250]	validation_0-rmse:261.82631	validation_1-rmse:396.78694
[499]	validation_0-rmse:216.61684	validation_1-rmse:368.26798
[0]	validation_0-rmse:1009.63986	validation_1-rmse:1348.96648
[250]	validation_0-rmse:280.66451	validation_1-rmse:497.87581
[499]	validation_0-rmse:233.82003	validation_1-rmse:466.80094
CPU times: user 2h 6min 44s, sys: 5.86 s, total: 2h 6min 50s
Wall time: 33min

In [51]:
# Predict submission data set
submission_df['prediction'] = reg_model.predict(submission_df)

In [52]:
submission_df

,store_nbr,family,onpromotion,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,lag16days,lag21days,lag28days,type,locale,locale_name,description,prediction
date,,,,,,,,,,,,,,,,,,
2017-08-16,1,0,0,2,3,8,2017,228,16,33,8.000,2.00000,7.000,3,2,6,8,8.580265
2017-08-16,1,1,0,2,3,8,2017,228,16,33,0.000,0.00000,0.000,3,2,6,8,4.197167
2017-08-16,1,2,2,2,3,8,2017,228,16,33,3.000,3.00000,3.000,3,2,6,8,128.926620
2017-08-16,1,3,20,2,3,8,2017,228,16,33,2414.000,2242.00000,2369.000,3,2,6,8,2379.531738
2017-08-16,1,4,0,2,3,8,2017,228,16,33,1.000,0.00000,0.000,3,2,6,8,4.737105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-31,9,28,1,3,3,8,2017,243,31,35,438.133,291.82098,425.946,3,2,6,8,349.035522
2017-08-31,9,29,0,3,3,8,2017,243,31,35,154.553,111.93000,83.426,3,2,6,8,99.820541
2017-08-31,9,30,1,3,3,8,2017,243,31,35,2419.729,1036.43900,1364.578,3,2,6,8,1290.692383


In [53]:
submission_df['prediction'].describe()

count    28512.000000
mean       472.233276
std       1213.146362
min         -2.773923
25%          4.846279
50%         56.797537
75%        291.779099
max      13421.495117
Name: prediction, dtype: float64

In [54]:
# Drop sales column 
sample_submission.drop('sales', inplace=True, axis=1)

# Data frame with only prediction values
t = pd.DataFrame(submission_df['prediction'])

# change DateTime index to integer
t.reset_index(inplace=True)

# add new conlumn sales with the prediction values
sample_submission['sales'] = t['prediction']

In [55]:
sample_submission

,id,sales
0,3000888,8.580265
1,3000889,4.197167
2,3000890,128.926620
3,3000891,2379.531738
4,3000892,4.737105
...,...,...
28507,3029395,349.035522
28508,3029396,99.820541
28509,3029397,1290.692383
28510,3029398,132.808304


In [56]:
sample_submission.to_csv('submission.csv', index=False)